### Messages

Messages are the fundamental unit of context for models in Langchain.They represent the input and output of models.

carrying both the content and metadataa needed to represent the state of a conversation when interacting with a LLM.

Messages are the objects that contain:

1-Role: Identify the message type (e.g. system,user)                                                                                   
2-Content: Represents the actual conent of the message (like text,images,audio,documents etc).                                        
3-Metadata: optional field such as response information,message IDs and token usage

Langchain provides a standard message type that works across all the model providers,ensuring consistent behavior regardless of the model being called.

In [24]:
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

model=init_chat_model("groq:llama-3.1-8b-instant")
model.invoke("Hello, how are you?")

AIMessage(content="I'm functioning properly, thank you for asking. I'm a large language model, so I don't have feelings in the same way that humans do, but I'm here to help with any questions or tasks you may have. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 54, 'prompt_tokens': 41, 'total_tokens': 95, 'completion_time': 0.06669106, 'completion_tokens_details': None, 'prompt_time': 0.001963217, 'prompt_tokens_details': None, 'queue_time': 0.046456373, 'total_time': 0.068654277}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f757f4b0bf', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c900f-f273-7b62-907f-727afc53c1e0-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 41, 'output_tokens': 54, 'total_tokens': 95})

### Text prompt

Use text prompt when:                                                                                                                   
    1-you have a single standalone text                                                                                             
    2-you dont need conversation history                                                                                                
    3-you want minimal code complexity

#Message prompt

1-System Messgage                                                                                                                       
2-Human Message                                                                                                                         
3-AI Message

In [39]:
from langchain.messages import SystemMessage, HumanMessage,AIMessage


system_message = SystemMessage(content="You are a judge. The user name is Sangram")
human_message = HumanMessage(content="Hello, how are you?")

messages = [system_message, human_message]
metadata = {"user_id": "1234567890","user_name":"Sangram"}

response = model.invoke(
    messages,
    config={
        "metadata": metadata
    }
)
print(response.content)



Good day to you, Sangram. I am well, thank you for asking. I have a busy schedule ahead of me, with several cases to preside over today. But I am always happy to take a brief moment to speak with you. What brings you here today? Are you seeking some guidance or perhaps have a matter that requires my attention?


In [40]:
response.usage_metadata

{'input_tokens': 52, 'output_tokens': 72, 'total_tokens': 124}

### Tool Message

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain.messages import (
    SystemMessage,
    HumanMessage,
    ToolMessage
)

# 1️⃣ Define a tool
@tool
def add_numbers(a: int, b: int) -> int:
    """Add two numbers together."""
    return a + b


# 2️⃣ Create model with tool binding
model_with_tools = model.bind_tools([add_numbers])


# 3️⃣ Send user message
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is 5 + 3?")
]

# 4️⃣ Model decides to call tool
ai_msg = model_with_tools.invoke(messages)


print("AI tool call:", ai_msg.tool_calls)

# 5️⃣ Execute tool manually
tool_call = ai_msg.tool_calls[0]

result = add_numbers.invoke(tool_call["args"])

# 6️⃣ Send ToolMessage back to model
messages.append(ai_msg)

messages.append(
    ToolMessage(
        content=str(result),
        tool_call_id=tool_call["id"]
    )
)

# 7️⃣ Final model response
final_response = model_with_tools.invoke(messages)

print("Final Answer:", final_response.content)


AI tool call: [{'name': 'add_numbers', 'args': {'a': 5, 'b': 3}, 'id': 'pgcavpz82', 'type': 'tool_call'}]
Final Answer: The result of 5 + 3 is 8.
